In [ ]:
! pip install datasets transformers accelerate evaluate degender-pronoun

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset
from evaluate import evaluator
from degender_pronoun import degenderizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import pandas as pd

model_checkpoint = "ben-yu/roberta-base-finetuned-nlp-letters-TEXT-pronouns-class-weighted"

model_name = model_checkpoint.split("/")[-1]
num_labels = 2


degender_pronouns = {
    ' mr ': ' mx ',
    ' mrs ': ' mx ',
    ' ms ': ' mx ',
    ' miss ': ' mx ',
    ' mister ': ' mx ',
}

degender_nouns = {
    ' man ': ' person ',
    ' men ': ' persons ',
    ' woman ': ' person ',
    ' women ': ' persons ',
    " man's ": " person's",
    " men's ": " person's",
    " woman's ": " person's",
    " women's ": " person's",
    " gentleman ": " person ",
    " lady ": " person ",
    " gentleman's ": " person's ",
    " lady's ": " person's ",
}

def preprocess(df, data_column, preprocess_type):
    if preprocess_type == 'none':
        return df

    D = degenderizer()
    df[data_column] = df[data_column].apply(lambda x: D.degender(x) if len(x) > 5 else x)

    for k, v in degender_pronouns.items():
        df[data_column] = df[data_column].str.lower().replace(k,v)

    if preprocess_type == 'all':
        for k, v in degender_nouns.items():
            df[data_column] = df[data_column].str.lower().replace(k,v)
    return df

data_column = 'TEXT'
preprocessing_type = 'all'
dataset_path = "/content/drive/MyDrive/sentence_sets.csv"
df = pd.read_csv(dataset_path, encoding='unicode_escape')
df = preprocess(df, data_column, preprocessing_type)

dataset = Dataset.from_pandas(df).rename_column("APPLICANT_GENDER", "label").class_encode_column("label").train_test_split(test_size=0.2)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
task_evaluator = evaluator("text-classification")
results = task_evaluator.compute(
    model_or_pipeline=model,
    data=dataset['test'],
    metric="accuracy",
    input_column=data_column,
    label_column="label",
    label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0},
    tokenizer=tokenizer,
)

results

Casting to class labels:   0%|          | 0/4411 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'accuracy': 0.9626274065685164,
 'total_time_in_seconds': 13.70593146400006,
 'samples_per_second': 64.42466185675042,
 'latency_in_seconds': 0.015522006187995537}

In [ ]:
dataset = Dataset.from_pandas(df).rename_column("APPLICANT_GENDER", "label").class_encode_column("label").train_test_split(test_size=0.2)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
task_evaluator = evaluator("text-classification")
results = task_evaluator.compute(
    model_or_pipeline=model,
    data=dataset['test'],
    metric="f1",
    input_column=data_column,
    label_column="label",
    label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0},
    tokenizer=tokenizer,
)

results

Casting to class labels:   0%|          | 0/4411 [00:00<?, ? examples/s]

{'f1': 0.9696969696969696,
 'total_time_in_seconds': 12.863647147999473,
 'samples_per_second': 68.64305199302068,
 'latency_in_seconds': 0.014568116815401442}

In [ ]:
dataset['test'][0]

{'LETTER_NUMBER': 4,
 'WRITER_GENDER': 'FEMALE',
 'LETTER_TYPE': 'Dept. Chair LoR',
 'label': 1,
 'APPLICANT_IDENTIFY': 'White',
 'CODED_ID': 956,
 'TEXT': 'it is my privilege to write this letter of support for first_name last_name , one of my most superb medical students at the keck school of medicine . first_name joined my clinical lab during their second year of medical school . they has been extremely productive , has published two peer reviewed manuscripts and presented at three national meetings . they is poised on the podium and can answer questions succinctly and completely . first_name has outstanding work ethic and has always been a reliable research assistant . when there are abstract or manuscript deadlines , they is awake at all hours of the day and night to make sure they are submitted . first_name is fueled by their desire to pursue interesting and relevant clinical research , and we speak almost daily on their ideas which i encourage them to pursue . they has an excell

In [ ]:
df = pd.read_csv(dataset_path, encoding='unicode_escape')
df = preprocess(df, data_column, preprocessing_type)
df

,LETTER_NUMBER,WRITER_GENDER,LETTER_TYPE,APPLICANT_GENDER,APPLICANT_IDENTIFY,CODED_ID,TEXT,STANDARDIZED_LOR,US_CANADIAN,USMLE_1,USMLE_2,S1,S2,S3,s1_s2
0,1,MALE,LoR,MALE,White,1,first_name last_name exceeded expectations in ...,1,1.0,227.0,249.0,FIRST_NAME LAST_NAME * he was described by his...,he completed all rotations receiving exception...,exceeded expectations in all standards and com...,FIRST_NAME LAST_NAME * he was described by his...
1,2,MALE,LoR,MALE,White,1,it is my honor and privilege to be able to wri...,0,1.0,227.0,249.0,it is my honor and privilege to be able to wri...,I can confidently state that he is the most co...,md .,it is my honor and privilege to be able to wri...
2,3,MALE,LoR,MALE,White,1,it is a great pleasure for me to write a lette...,0,1.0,227.0,249.0,it is a great pleasure for me to write a lette...,"he worked problems out to completion , and pre...",NaN,it is a great pleasure for me to write a lette...
3,4,MALE,PD LoR,MALE,White,1,it is my pleasure to write this letter of reco...,0,1.0,227.0,249.0,it is my pleasure to write this letter of reco...,", long number in support of his application f...","as a result , I have reviewed and read , in de...",it is my pleasure to write this letter of reco...
4,1,FEMALE,LoR,MALE,White,2,this letter is written on behalf of first_name...,0,1.0,NaN,NaN,this letter is written on behalf of FIRST_NAME...,do for consideration of his acceptance into yo...,"should you desire additional information , ple...",this letter is written on behalf of FIRST_NAME...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,4,MALE,Dept. Chair LoR,MALE,Asian - Indian,1235,it ts my great pleasure to write a letter of r...,0,0.0,241.0,253.0,it ts my great pleasure to write a letter of ...,"is a medical graduate from india , had the opp...",yours sincerely,it ts my great pleasure to write a letter of ...
4407,1,MALE,LoR,MALE,"Hispanic, Latino, or of Spanish origin - Other...",1236,it is my pleasure to provide a letter of recom...,0,0.0,233.0,219.0,it is my pleasure to provide a letter of recom...,he was an outstanding student and received an ...,NaN,it is my pleasure to provide a letter of recom...
4408,2,MALE,LoR,MALE,"Hispanic, Latino, or of Spanish origin - Other...",1236,it is with great enthusiasm that i write this ...,0,0.0,233.0,219.0,it is with great enthusiasm that i write this ...,from among many distinguished applicants to re...,trainees .,it is with great enthusiasm that i write this ...
4409,3,MALE,LoR,MALE,"Hispanic, Latino, or of Spanish origin - Other...",1236,it is my pleasure to strongly support first_na...,0,0.0,233.0,219.0,it is my pleasure to strongly support FIRST_NA...,while he was a graduate student . * he joined ...,dissertation project under perenlei enkhbaatar...,it is my pleasure to strongly support FIRST_NA...


In [ ]:
df.iloc[9]['TEXT']

"l am a pediatric anesthesiologist at children's hospital colorado within the university of colorado medical system . i frequently work with trainees of all levels in the operating room and had the opportunity to work with first_name ( middle_name ) last_name in the spring of long number middle_name is one of the most engaged and intelligent medical students i have had the pleasure to work with . i am writing to give them my enthusiastic recommendation for acceptance into your anesthesia training program . middle_name is a non - traditional student who has taken a long and indirect path to arrive at this point in their career . their studies have included music , environmental sciences and , finally , medicine . at each stage of their career they demonstrated proficiency within their field , including earning a phd and completing post - doctorate work . importantly , they approached each change in their career path with a thoughtfulness and self - awareness that has allowed them to del

In [ ]:
!pip install scikit-learn==1.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 73.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.11 requires scikit-learn>=1.3.2, but you have scikit-learn 1.1.0 which is incompatible.
bigframes 1.11.1 requires scikit-learn>=1.2.2, but you have scikit-learn 1.1.0 which is incompatible.


In [ ]:
import sys, gc
import scipy
import numpy as np
def monkeypath_itemfreq(sampler_indices):
   return zip(*np.unique(sampler_indices, return_counts=True))

scipy.stats.itemfreq=monkeypath_itemfreq

import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from eli5.lime import TextExplainer
from eli5.formatters import format_as_dict
from degender_pronoun import degenderizer
from datasets import Dataset


import pandas as pd

degender_pronouns = {
    ' mr ': ' mx ',
    ' mrs ': ' mx ',
    ' ms ': ' mx ',
    ' miss ': ' mx ',
    ' mister ': ' mx ',
}

degender_nouns = {
    ' man ': ' person ',
    ' men ': ' persons ',
    ' woman ': ' person ',
    ' women ': ' persons ',
    " man's ": " person's",
    " men's ": " person's",
    " woman's ": " person's",
    " women's ": " person's",
    " gentleman ": " person ",
    " lady ": " person ",
    " gentleman's ": " person's ",
    " lady's ": " person's ",
}

def preprocess(df, data_column, preprocess_type):
    if preprocess_type == 'none':
        return df

    D = degenderizer()
    df[data_column] = df[data_column].apply(lambda x: D.degender(x) if len(x) > 5 else x)

    for k, v in degender_pronouns.items():
        df[data_column] = df[data_column].str.lower().replace(k,v)

    if preprocess_type == 'all':
        for k, v in degender_nouns.items():
            df[data_column] = df[data_column].str.lower().replace(k,v)
    return df

model_checkpoint = "ben-yu/roberta-base-finetuned-nlp-letters-TEXT-pronouns-class-weighted"
batch_size = 16
task = "nlp-letters"
metric_name = "accuracy"
data_column = "full_text"
model_name = model_checkpoint.split("/")[-1]
num_labels = 2

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels, low_cpu_mem_usage=True)
def model_adapter(texts):

    all_scores = []
    for i in range(0, len(texts), 64):
        batch = texts[i:i+64]

        # use bert encoder to tokenize text
        encoded_input = tokenizer(batch,
          return_tensors='pt',
          padding=True,
          truncation=True,
          max_length=model.config.max_position_embeddings-2)
        # run the model
        output = model(**encoded_input)
        # by default this model gives raw logits rather
        # than a nice smooth softmax so we apply it ourselves here
        scores = output[0].softmax(1).detach().numpy()
        all_scores.extend(scores)
    return np.array(all_scores)


class_names = ['female', 'male']
te = TextExplainer(n_samples=5000, random_state=42, position_dependent=True)

data_column = 'TEXT'
preprocessing_type = 'all'
dataset_path = "/content/drive/MyDrive/sentence_sets.csv"
df = pd.read_csv(dataset_path, encoding='unicode_escape')
df = preprocess(df, data_column, preprocessing_type)
dataset = Dataset.from_pandas(df).rename_column("APPLICANT_GENDER", "label").class_encode_column("label").train_test_split(test_size=0.2)


for data in dataset['test']:
  text = data['TEXT']
  gc.collect()
  te.fit(text, model_adapter)
  explain = te.explain_weights(target_names=class_names)
  print(format_as_dict(explain))

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Casting to class labels:   0%|          | 0/4411 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


{'estimator': "SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x7AD3EA997540)", 'description': "\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", 'error': None, 'method': 'linear model', 'is_regression': False, 'targets': [{'target': 'male', 'feature_weights': {'pos': [{'feature': '<BIAS>', 'weight': 3.862008104414173, 'std': None, 'value': None}, {'feature': '[215] deficiencies', 'weight': 3.31044101683658, 'std': None, 'value': None}, {'feature': '[173] them', 'weight': 0.470278481

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


{'estimator': "SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x7AD3EA997840)", 'description': "\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", 'error': None, 'method': 'linear model', 'is_regression': False, 'targets': [{'target': 'male', 'feature_weights': {'pos': [{'feature': '<BIAS>', 'weight': 4.255588010470537, 'std': None, 'value': None}, {'feature': '[252] knowledge', 'weight': 3.4077348018272833, 'std': None, 'value': None}, {'feature': '[110] them', 'weight': 0.6728045865

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


{'estimator': "SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x7AD3EA997540)", 'description': "\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", 'error': None, 'method': 'linear model', 'is_regression': False, 'targets': [{'target': 'male', 'feature_weights': {'pos': [{'feature': '<BIAS>', 'weight': 2.937849805988707, 'std': None, 'value': None}, {'feature': '[242] peers', 'weight': 1.0095870743747342, 'std': None, 'value': None}, {'feature': '[315] them', 'weight': 0.72180600689099

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


{'estimator': "SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x7AD3EA997840)", 'description': "\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", 'error': None, 'method': 'linear model', 'is_regression': False, 'targets': [{'target': 'male', 'feature_weights': {'pos': [{'feature': '<BIAS>', 'weight': 2.7553412284609737, 'std': None, 'value': None}, {'feature': '[213] experiences', 'weight': 1.2598762698673798, 'std': None, 'value': None}, {'feature': '[161] interest', 'weight': 1.195

In [ ]:
import scipy
import numpy as np
def monkeypath_itemfreq(sampler_indices):
   return zip(*np.unique(sampler_indices, return_counts=True))

scipy.stats.itemfreq=monkeypath_itemfreq

import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from eli5.lime import TextExplainer




model_checkpoint = "ben-yu/roberta-base-finetuned-nlp-letters-TEXT-pronouns-class-weighted"
batch_size = 16
task = "nlp-letters"
metric_name = "accuracy"
data_column = "full_text"
model_name = model_checkpoint.split("/")[-1]
num_labels = 2

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels, low_cpu_mem_usage=True)
def model_adapter(texts):

    all_scores = []
    for i in range(0, len(texts), 64):
        batch = texts[i:i+64]

        # use bert encoder to tokenize text
        encoded_input = tokenizer(batch,
          return_tensors='pt',
          padding=True,
          truncation=True,
          max_length=model.config.max_position_embeddings-2)
        # run the model
        output = model(**encoded_input)
        # by default this model gives raw logits rather
        # than a nice smooth softmax so we apply it ourselves here
        scores = output[0].softmax(1).detach().numpy()
        all_scores.extend(scores)
    return np.array(all_scores)


class_names = ['female', 'male']
text = """
I am pleased to provide my unqualified support for FIRST_NAME LAST_NAME  FIRST_NAME strikes me as someone who  will achieve at a high level in a residency program  she is mature   has other non   medical experiences   teach for america     and has a demonstrated record of superior achievement  her board scores are competitive and she has numerous honors in our major clinical rotations  I have spent time in conversation with her regarding anesthesiology and I have worked with her in the operating room  in the operating room   I was particularly struck by her aptitude and quiet confidence  I hope you ask her about her experiences while a teacher  I find these supplemental life experiences additive to any program  I know that we will make every effort to recruit her to unc
"""

te = TextExplainer(n_samples=5000, random_state=42, position_dependent=True)
te.fit(D.degender(text), model_adapter)
te.explain_prediction(target_names=class_names)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Explanation(estimator="SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x788A0E328D40)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='male', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[102] experiences', weight=1.530778304168425, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=0.7182951095265415, std=None, value=1.0), FeatureWeight(feature='[91] aptitude', weight=0.5576194059603512, std=None, value=1.0), FeatureWeight(feature='[94] confidence', weight=0.36479520871030097, std=None, value=1.0), FeatureWeight(feature='[105] teacher', weight=0.30907863487491716, std=None, value=1.0), FeatureWeight(feature='[99] them', weight=0.3079141951282917, std=None, value=1.0), FeatureWeight(feature='[68] them', weight=0.3067636927494124, std=None, value=1.0), FeatureWeight(feature='[76] them', weight=0.3008217273648056, std=None, value=1.0), FeatureWeight(feature='[126] them', weight=0.24144782614566576, std=None, value=1.0), FeatureWeight(feature='[47] board', weight=0.16562015038581648, std=None, value=1.0), FeatureWeight(feature='[93] quiet', weight=0.15240310833262283, std=None, value=1.0), FeatureWeight(feature='[44] superior', weight=0.13613379112133336, std=None, value=1.0), FeatureWeight(feature='[48] scores', weight=0.12410024773758638, std=None, value=1.0), FeatureWeight(feature='[32] non', weight=0.1192311957994065, std=None, value=1.0), FeatureWeight(feature='[6] unqualified', weight=0.10338309313855015, std=None, value=1.0), FeatureWeight(feature='[18] achieve', weight=0.10146005103669935, std=None, value=1.0), FeatureWeight(feature='[3] to', weight=0.09923853001662065, std=None, value=1.0), FeatureWeight(feature='[74] worked', weight=0.09675935337501386, std=None, value=1.0), FeatureWeight(feature='[10] LAST_NAME', weight=0.07846351469478571, std=None, value=1.0), FeatureWeight(feature='[50] competitive', weight=0.07777097045107169, std=None, value=1.0), FeatureWeight(feature='[96] hope', weight=0.07763236708660394, std=None, value=1.0), FeatureWeight(feature='[23] in', weight=0.07514926364330439, std=None, value=1.0), FeatureWeight(feature='[85] I', weight=0.07483320957424275, std=None, value=1.0), FeatureWeight(feature='[67] with', weight=0.06266048676212424, std=None, value=1.0), FeatureWeight(feature='[27] they', weight=0.057587101617516026, std=None, value=1.0), FeatureWeight(feature='[118] that', weight=0.0560240139139093, std=None, value=1.0), FeatureWeight(feature='[5] my', weight=0.05491082401029797, std=None, value=1.0), FeatureWeight(feature='[63] spent', weight=0.05102521587575609, std=None, value=1.0), FeatureWeight(feature='[56] in', weight=0.047958262093157976, std=None, value=1.0), FeatureWeight(feature='[109] supplemental', weight=0.045091194700456076, std=None, value=1.0), FeatureWeight(feature='[51] and', weight=0.04471744283872262, std=None, value=1.0), FeatureWeight(feature='[123] effort', weight=0.04118150211196058, std=None, value=1.0), FeatureWeight(feature='[24] a', weight=0.04106799285591208, std=None, value=1.0), FeatureWeight(feature='[66] conversation', weight=0.039551731827938585, std=None, value=1.0), FeatureWeight(feature='[114] any', weight=0.039371506879755803, std=None, value=1.0), FeatureWeight(feature='[35] teach', weight=0.033669514288722564, std=None, value=1.0), FeatureWeight(feature='[115] program', weight=0.025975554893814663, std=None, value=1.0), FeatureWeight(feature='[40] a', weight=0.02318621557322104, std=None, value=1.0), FeatureWeight(feature='[121] make', weight=0.02157763489145755, std=None, value=1.0), FeatureWeight(feature='[120] will', weight=0.020347869204141902, std=None, value=1.0), FeatureWeight(feature='[22] level', weight=0.01608106179327378, std=None, value=1.0), FeatureWeight(feature='[116] I', weight=0.00942199408800239, std=None, value=1.0), FeatureWeight(feature='[82] the', weight=0.007271563603004553, std=

Weight?,Feature
+1.531,[102] experiences
+0.718,<BIAS>
+0.558,[91] aptitude
+0.365,[94] confidence
+0.309,[105] teacher
+0.308,[99] them
+0.307,[68] them
+0.301,[76] them
+0.241,[126] them
+0.166,[47] board


In [ ]:
class_names = ['female', 'male']
text = """
this letter serves as the department of medicine letter for FIRST_NAME LAST_NAME  these letters are written and compiled by the clerkship director and associate clerkship director with help from our clerkship coordinator  they are written in accordance with the cdim   apdim guidelines for department of medicine letters  LAST_NAME waived his right to review this letter  the core internal medicine clerkship at unm is 8   weeks in duration and consists of 6 weeks spent in the inpatient setting at our university and veteran's affairs hospitals as well as 2 weeks split on palliative care and ambulatory medicine  students final grade is on the ogsf scale   outstanding   good   satisfactory   fail   with 50 % of the grade based on clinical performance   25 % on shelf board   15 % on quizzes   consisting of students writing a problem list   assessment statement and differential diagnosis based on a paper case     and 10 % on professionalism  LAST_NAME received a clinical score of outstanding   a shelf board score of satisfactory   and an overall core im clerkship score of good  LAST_NAME received these comments on his core im clerkship    great documentation with accurate and well thought out assessments  active in requesting new tasks to help team and patient care  î attending e FIRST_NAME performed well on the rotation   specific strengths included actively seeking opportunities for patient care   using strong communication strategies to engender trust with patients   asking for continuous feedback and seeking opportunities to educate team   opportunities infections in hiv      attending e it was a pleasure working with FIRST_NAME on wards  FIRST_NAME's a team player   gets along well with other staff and patients and family  FIRST_NAME helped with team work and was active member of the team  his presentations were detailed oriented   write ups were well organized and thoughtful  he took extra initiative in educating the team about various topics including atrial fibrillation management   relevance of mrsa nares   sleep medications and medication interactions  he stayed late on several occasions to finish his work and help the team  he commits to patient care and delivers  î   attending  great patient care   prioritizes tasks   gathers all pertinent information from patients  outstanding knowledge   great clinical correlation   identifies new problems independently  great patient ownership   even followed important aspects of patients that he saw before but not seeing that day  really great bedside manner  spontaneously researches on patients cases and educates the team with what he learns  FIRST_NAME has confidence in himself and enjoys the profession he is about to practice   he has a great attitude   that makes working with him to be fun  I personally learned from his great positive attitude and disposition  professional with patients and team  î resident e sees a lot of patients   takes initiative to see new patients   good communication skills with patients   works well with residents   open to feedback  î resident in summary   FIRST_NAME completed internal medicine as his first clerkship in his phase il experience and performed very well clinically  he showed a strong work ethic during his clerkship within internal medicine   being noted by both unmh and va attendings to be an active member of the team and stay late when needed  he will FIRST_NAME a strong resident physician """

te = TextExplainer(n_samples=5000, random_state=42, position_dependent=True)
te.fit(D.degender(text), model_adapter)
te.explain_prediction(target_names=class_names)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Explanation(estimator="SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x788A10C7ED40)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='female', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[305] mrsa', weight=0.8340101452390452, std=None, value=1.0), FeatureWeight(feature='[414] to', weight=0.7248493525849338, std=None, value=1.0), FeatureWeight(feature='[306] nares', weight=0.3482979646264852, std=None, value=1.0), FeatureWeight(feature='[324] the', weight=0.3002504274449922, std=None, value=1.0), FeatureWeight(feature='[421] their', weight=0.26166565825912635, std=None, value=1.0), FeatureWeight(feature='[45] medicine', weight=0.2264725582101421, std=None, value=1.0), FeatureWeight(feature='[467] their', weight=0.21848489354821227, std=None, value=1.0), FeatureWeight(feature='[322] and', weight=0.1977552317796821, std=None, value=1.0), FeatureWeight(feature='[33] are', weight=0.19642071137898628, std=None, value=1.0), FeatureWeight(feature='[384] the', weight=0.193416664984898, std=None, value=1.0), FeatureWeight(feature='[236] in', weight=0.17697112486187963, std=None, value=1.0), FeatureWeight(feature='[302] management', weight=0.17290311585037632, std=None, value=1.0), FeatureWeight(feature='[482] a', weight=0.1701913855565919, std=None, value=1.0), FeatureWeight(feature='[386] with', weight=0.15692635303995917, std=None, value=1.0), FeatureWeight(feature='[276] their', weight=0.15593421636415308, std=None, value=1.0), FeatureWeight(feature='[433] resident', weight=0.14784042076621245, std=None, value=1.0), FeatureWeight(feature='[257] with', weight=0.14632250559739227, std=None, value=1.0), FeatureWeight(feature='[244] working', weight=0.14572495321160417, std=None, value=1.0), FeatureWeight(feature='[328] to', weight=0.13843778431913675, std=None, value=1.0), FeatureWeight(feature='[188] requesting', weight=0.13840276095507253, std=None, value=1.0), FeatureWeight(feature='[265] helped', weight=0.13737746109434162, std=None, value=1.0), FeatureWeight(feature='[78] our', weight=0.13713975752737387, std=None, value=1.0), FeatureWeight(feature='[230] opportunities', weight=0.13555667354369513, std=None, value=1.0), FeatureWeight(feature='[101] on', weight=0.13527022066195607, std=None, value=1.0), FeatureWeight(feature='[148] a', weight=0.13465043442047928, std=None, value=1.0), FeatureWeight(feature='[383] educates', weight=0.1346408348568013, std=None, value=1.0), FeatureWeight(feature='[471] their', weight=0.12164493395089185, std=None, value=1.0), FeatureWeight(feature='[323] help', weight=0.12152428619546943, std=None, value=1.0), FeatureWeight(feature='[247] on', weight=0.11851272489019309, std=None, value=1.0), FeatureWeight(feature='[57] internal', weight=0.11449358982430467, std=None, value=1.0), FeatureWeight(feature='[266] with', weight=0.11376333436915957, std=None, value=1.0), FeatureWeight(feature='[422] great', weight=0.11227032407897125, std=None, value=1.0), FeatureWeight(feature='[333] î', weight=0.11218414388731181, std=None, value=1.0), FeatureWeight(feature='[183] thought', weight=0.11192244428796441, std=None, value=1.0), FeatureWeight(feature='[283] were', weight=0.11111244381783822, std=None, value=1.0), FeatureWeight(feature='[116] clinical', weight=0.109852230137108, std=None, value=1.0), FeatureWeight(feature='[134] and', weight=0.10721157893684381, std=None, value=1.0), FeatureWeight(feature='[497] and', weight=0.1072044161512684, std=None, value=1.0), FeatureWeight(feature='[477] very', weight=0.10527856208840461, std=None, value=1.0), FeatureWeight(feature='[240] it', weight=0.10499223378426356, std=None, value=1.0), FeatureWeight(feature='[377] spontaneously', weight=0.10291399759165848, std=None, value=1.0), FeatureWeight(feature='[393] in', weight=0.10222449521910287, std=None, value=1.0), FeatureWeight(feature='[51] to', weight=0.10141110062280431, st

In [ ]:
te.explain_weights(target_names=class_names)

Weight?,Feature
+1.653,<BIAS>
+0.450,[413] them
+0.322,[385] team
+0.281,[301] fibrillation
+0.276,[469] clerkship
+0.248,[424] attitude
+0.245,[474] experience
+0.239,[325] team
+0.212,[237] hiv
+0.189,[321] work
